
# REST vs MCP vs A2A — End‑to‑End Demo (Single Notebook)

This notebook shows a **working** example that you can run locally:

1. **REST API** with FastAPI (served inside the notebook)
2. **MCP‑style tool calls** from an "LLM" to functions you expose in Python
3. **Agent‑to‑Agent (A2A)** workflow (DataCollector ↔ Summarizer)
4. **Combined Architecture** — REST endpoint that triggers the A2A workflow, and a minimal MCP call

> ✅ The notebook uses lightweight, built‑in code where possible, and only **FastAPI**, **uvicorn**, and **httpx** are added (they often come preinstalled in many environments). If you cannot install packages, you can still read the code and run the A2A + MCP sections which have zero external deps.



## 0) Setup — Install Light Dependencies (FastAPI + Uvicorn + httpx)

If these are already installed in your environment, the cells will skip re-installing.


In [1]:
import nest_asyncio
import uvicorn

# Removed uvicorn.run(app) as the server is started in a separate thread later.
nest_asyncio.apply()

In [2]:

import sys, subprocess, pkgutil

def ensure(pkg, pip_name=None):
    pip_name = pip_name or pkg
    if pkg in [m.name for m in pkgutil.iter_modules()]:
        print(f"{pkg} already installed.")
        return
    print(f"Installing {pip_name}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pip_name])

try:
    ensure("fastapi")
    ensure("uvicorn")
    ensure("httpx")
except Exception as e:
    print("⚠️ Install step failed. You can still run MCP + A2A sections without FastAPI.")
    print(e)


fastapi already installed.
uvicorn already installed.
httpx already installed.


In [3]:
import threading, time
import contextlib

# nest_asyncio avoids "already running event loop" issues in notebooks
try:
    import nest_asyncio
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "nest_asyncio"])
    import nest_asyncio

nest_asyncio.apply()

from fastapi import FastAPI, HTTPException
from typing import Dict
import uvicorn

app = FastAPI(title="Notebook REST API")

FAKE_WEATHER: Dict[str, Dict] = {
    "London": {"temp_c": 18, "condition": "Partly cloudy"},
    "Dublin": {"temp_c": 16, "condition": "Showers"},
    "Paris":  {"temp_c": 20, "condition": "Sunny"},
}

@app.get("/weather")
def get_weather(city: str):
    data = FAKE_WEATHER.get(city)
    if not data:
        raise HTTPException(status_code=404, detail=f"No data for city={city!r}")
    return {"city": city, **data}

# Start server in background
server = None
def run_server():
    global server
    config = uvicorn.Config(app, host="127.0.0.1", port=8008, log_level="warning")
    server = uvicorn.Server(config)
    server.run()

srv_thread = threading.Thread(target=run_server, daemon=True)
srv_thread.start()

# Give server a moment
time.sleep(1.0)
print("✅ FastAPI server should be running on http://127.0.0.1:8008")

✅ FastAPI server should be running on http://127.0.0.1:8008



## 1) REST API — Minimal FastAPI Service (Run In-Notebook)

We'll spin up a **FastAPI** app inside the notebook using a background thread.  
Endpoint: `GET /weather?city=London` → returns JSON.

Then we'll call it from the notebook using **httpx**.


In [4]:

# Client call to our REST API using httpx
import httpx, json

try:
    r = httpx.get("http://127.0.0.1:8008/weather", params={"city":"London"}, timeout=5.0)
    print("Status:", r.status_code)
    print(json.dumps(r.json(), indent=2))
except Exception as e:
    print("⚠️ Could not reach the server:", e)
    print("If you're running this somewhere that blocks local servers, skip to the next section.")


Status: 200
{
  "city": "London",
  "temp_c": 18,
  "condition": "Partly cloudy"
}



> To stop the server (optional), run the next cell.


In [5]:

# Graceful stop helper (best-effort; server thread is daemonized)
print("Server thread daemonized; it will stop when the kernel stops. No action required.")


Server thread daemonized; it will stop when the kernel stops. No action required.



## 2) Minimal MCP‑Style Tool Invocation (No External Deps)

**Goal:** mimic the spirit of MCP: the model calls **declared tools** with structured arguments, gets structured results back.

We'll define:
- a **ToolRegistry** with schema and callables
- a fake **LLM** that decides which tool to call (we'll simulate choice)
- a **`get_weather`** tool (re-using our in-notebook data)

This isn't a full MCP spec; it's a compact demo you can extend.


In [6]:

from typing import Any, Callable, Dict, List, Optional, Tuple

class ToolError(Exception):
    pass

class ToolRegistry:
    def __init__(self):
        self.tools: Dict[str, Tuple[Callable, Dict[str, Any]]] = {}  # name -> (func, schema)

    def register(self, name: str, func: Callable, schema: Dict[str, Any]):
        self.tools[name] = (func, schema)

    def describe(self) -> Dict[str, Any]:
        return {"tools": {name: schema for name, (_, schema) in self.tools.items()}}

    def call(self, name: str, args: Dict[str, Any]) -> Any:
        if name not in self.tools:
            raise ToolError(f"Unknown tool: {name}")
        func, schema = self.tools[name]
        # naive arg validation
        required = [p["name"] for p in schema.get("parameters", []) if p.get("required")]
        missing = [r for r in required if r not in args]
        if missing:
            raise ToolError(f"Missing required args: {missing}")
        return func(**args)

registry = ToolRegistry()

def tool_get_weather(city: str) -> Dict[str, Any]:
    if city not in FAKE_WEATHER:
        return {"ok": False, "error": f"No data for {city!r}"}
    return {"ok": True, "data": {"city": city, **FAKE_WEATHER[city]}}

registry.register(
    name="get_weather",
    func=tool_get_weather,
    schema={
        "description": "Returns current weather for a known city (demo).",
        "parameters": [
            {"name": "city", "type": "string", "required": True, "enum": list(FAKE_WEATHER.keys())}
        ],
        "returns": {"type": "object", "properties": {"ok": "bool", "data": "object"}},
    }
)

# "LLM" decides which tool to call based on a user prompt (simple rule-based demo)
def llm_route_and_call(user_prompt: str) -> Any:
    if "weather" in user_prompt.lower():
        # pick a city (very naive)
        city = None
        for c in FAKE_WEATHER.keys():
            if c.lower() in user_prompt.lower():
                city = c
                break
        city = city or "Dublin"
        return {"called": "get_weather", "result": registry.call("get_weather", {"city": city})}
    return {"called": None, "result": "I don't know which tool to use."}

print("Available tools:", registry.describe())
print(llm_route_and_call("What's the weather in Paris?"))


Available tools: {'tools': {'get_weather': {'description': 'Returns current weather for a known city (demo).', 'parameters': [{'name': 'city', 'type': 'string', 'required': True, 'enum': ['London', 'Dublin', 'Paris']}], 'returns': {'type': 'object', 'properties': {'ok': 'bool', 'data': 'object'}}}}}
{'called': 'get_weather', 'result': {'ok': True, 'data': {'city': 'Paris', 'temp_c': 20, 'condition': 'Sunny'}}}



## 3) Agent‑to‑Agent (A2A) Workflow

We'll create two lightweight agents:
- **DataCollector**: generates a small EV sales dataset (mock)
- **Summarizer**: reads the data and produces a natural language summary

They'll "chat" by passing a shared context dict.


In [7]:

import random
from statistics import mean

class Agent:
    def __init__(self, name: str):
        self.name = name

    def handle(self, context: Dict[str, Any]) -> Dict[str, Any]:
        raise NotImplementedError

class DataCollector(Agent):
    def handle(self, context):
        # Create mock EV sales data: year -> sales
        years = list(range(2020, 2025))
        data = [{"year": y, "ev_sales": random.randint(100_000, 500_000)} for y in years]
        context["ev_sales_data"] = data
        context.setdefault("log", []).append(f"{self.name}: collected {len(data)} rows.")
        return context

class Summarizer(Agent):
    def handle(self, context):
        data = context.get("ev_sales_data", [])
        if not data:
            context.setdefault("log", []).append(f"{self.name}: no data to summarize.")
            return context
        growth = data[-1]["ev_sales"] - data[0]["ev_sales"]
        avg = int(mean([d["ev_sales"] for d in data]))
        summary = (
            f"EV sales grew by {growth:,} from {data[0]['year']} to {data[-1]['year']}. "
            f"Average yearly sales ~ {avg:,}."
        )
        context["summary"] = summary
        context.setdefault("log", []).append(f"{self.name}: produced summary.")
        return context

def run_workflow():
    ctx: Dict[str, Any] = {}
    a = DataCollector("DataCollector")
    b = Summarizer("Summarizer")
    for agent in (a, b):
        ctx = agent.handle(ctx)
    return ctx

ctx = run_workflow()
print("Log:", *ctx["log"], sep=" | ")
print("Summary:", ctx.get("summary"))
print("Sample data:", ctx.get("ev_sales_data")[:2], "...")


Log: | DataCollector: collected 5 rows. | Summarizer: produced summary.
Summary: EV sales grew by -385,850 from 2020 to 2024. Average yearly sales ~ 270,759.
Sample data: [{'year': 2020, 'ev_sales': 485951}, {'year': 2021, 'ev_sales': 228428}] ...



## 4) Combined Architecture

We expose a REST endpoint `/ev/summary` that:
1. Runs the **A2A workflow** (DataCollector → Summarizer)
2. Returns the **summary text** and the data

We'll also show a minimal **MCP-style call** to fetch weather and combine it with the EV summary — just to demonstrate bridging tools + agents.


In [8]:

from fastapi import APIRouter
import json

router = APIRouter()

@router.get("/ev/summary")
def ev_summary():
    ctx = run_workflow()
    return {"summary": ctx.get("summary"), "rows": ctx.get("ev_sales_data", [])}

app.include_router(router)

print("✅ Added /ev/summary to the FastAPI app. Reusing the same server thread on :8008.")


✅ Added /ev/summary to the FastAPI app. Reusing the same server thread on :8008.


In [9]:

# Call the combined endpoint
try:
    r = httpx.get("http://127.0.0.1:8008/ev/summary", timeout=5.0)
    print("Status:", r.status_code)
    print(json.dumps(r.json(), indent=2))
except Exception as e:
    print("⚠️ Could not reach the server:", e)


Status: 200
{
  "summary": "EV sales grew by 128,666 from 2020 to 2024. Average yearly sales ~ 300,670.",
  "rows": [
    {
      "year": 2020,
      "ev_sales": 257384
    },
    {
      "year": 2021,
      "ev_sales": 172991
    },
    {
      "year": 2022,
      "ev_sales": 292259
    },
    {
      "year": 2023,
      "ev_sales": 394668
    },
    {
      "year": 2024,
      "ev_sales": 386050
    }
  ]
}



### 4.b) Minimal MCP‑Style + A2A Fusion

We'll fetch weather via the MCP‑style tool and append it to our EV summary.


In [10]:

def fused_report(city="Dublin"):
    # MCP-style tool call
    w = registry.call("get_weather", {"city": city})
    # A2A
    ctx = run_workflow()
    return {
        "weather": w,
        "ev_summary": ctx.get("summary"),
        "sample_rows": ctx.get("ev_sales_data", [])[:3]
    }

print(json.dumps(fused_report("London"), indent=2))


{
  "weather": {
    "ok": true,
    "data": {
      "city": "London",
      "temp_c": 18,
      "condition": "Partly cloudy"
    }
  },
  "ev_summary": "EV sales grew by -112,896 from 2020 to 2024. Average yearly sales ~ 281,883.",
  "sample_rows": [
    {
      "year": 2020,
      "ev_sales": 342381
    },
    {
      "year": 2021,
      "ev_sales": 449297
    },
    {
      "year": 2022,
      "ev_sales": 123408
    }
  ]
}



## 5) Where To Go Next

- Replace the mock weather with a real API (keep the REST wiring as is)
- Replace the toy MCP registry with an actual MCP server/client when available
- Swap the toy agents with frameworks (AutoGen, LangGraph/CrewAI) if desired
- Wrap your notebook as a service: keep FastAPI cells, deploy with `uvicorn main:app`
